# RAG 기본 구조 가이드

## 개요

**RAG(Retrieval-Augmented Generation)** 는 검색 기반 증강 생성 기술로, 외부 지식 소스를 활용하여 언어 모델의 응답 정확성과 신뢰성을 향상시키는 아키텍처입니다.

### RAG가 필요한 이유

일반적인 대형 언어 모델(LLM)은 다음과 같은 한계점을 가지고 있습니다:

| 문제점 | 설명 |
|-------|------|
| **지식 고정성** | 훈련 데이터 시점 이후의 최신 정보 부재 |
| **도메인 특화 지식 부족** | 조직 내부 문서나 전문 분야 자료에 대한 접근 제한 |
| **환각(Hallucination) 현상** | 사실과 다른 정보를 그럴듯하게 생성하는 문제 |
| **출처 불명확성** | 정보의 근거와 신뢰성 검증 어려움 |

### RAG 시스템의 핵심 원리

RAG는 **검색-증강-생성** 패러다임을 통해 이러한 문제를 해결합니다:

1. **외부 지식베이스 검색**: 사용자 질의와 관련된 문서를 실시간 검색
2. **맥락 증강**: 검색된 정보를 언어 모델의 입력 맥락에 추가
3. **증강된 생성**: 풍부한 맥락을 바탕으로 정확하고 근거 있는 응답 생성

## 학습 목표

이 튜토리얼을 통해 다음을 학습할 수 있습니다:

- RAG 파이프라인의 8단계 구현 과정
- PDF 문서를 활용한 실제 RAG 시스템 구축
- 벡터 데이터베이스와 임베딩 기술의 실무 활용
- LangChain을 이용한 RAG 체인 구성 및 최적화

# RAG 시스템 아키텍처

## RAG 파이프라인 구성

RAG 시스템은 **사전 처리(Pre-processing)** 와 **실시간 처리(Runtime)** 두 단계로 구분됩니다.

### 사전 처리 단계 (1-4단계)

![rag-1.png](./assets/rag-1.png)

![rag-1-graphic](./assets/rag-graphic-1.png)

지식베이스 구축을 위한 준비 과정입니다.

| 단계 | 프로세스 | 목적 | 기술 구현 |
|------|---------|------|----------|
| **1단계** | **문서 로드** | 다양한 형태의 문서를 시스템으로 가져오기 | PyMuPDFLoader, TextLoader 등 |
| **2단계** | **텍스트 분할** | 문서를 검색 가능한 청크 단위로 분할 | RecursiveCharacterTextSplitter |
| **3단계** | **임베딩 생성** | 텍스트를 벡터 공간의 수치 표현으로 변환 | OpenAI Embeddings, HuggingFace Embeddings |
| **4단계** | **벡터DB 저장** | 임베딩 벡터를 고속 검색 가능한 형태로 저장 | FAISS, Chroma, Pinecone |

#### 텍스트 분할 고려사항

- **Chunk Size**: 너무 작으면 맥락 손실, 너무 크면 검색 정확도 저하
- **Overlap**: 청크 간 중복 영역으로 맥락 연속성 보장
- **분할 방식**: 문서 구조(문단, 문장, 토큰)에 따른 적절한 분할 전략

### 실시간 처리 단계 (5-8단계)

![rag-2.png](./assets/rag-2.png)

![](./assets/rag-graphic-2.png)

사용자 질의에 대한 실시간 응답 생성 과정입니다.

| 단계 | 프로세스 | 목적 | 기술 구현 |
|------|---------|------|----------|
| **5단계** | **문서 검색** | 질의와 의미적으로 유사한 문서 청크 검색 | Similarity Search, Hybrid Search |
| **6단계** | **프롬프트 구성** | 검색된 맥락과 질의를 결합한 프롬프트 생성 | PromptTemplate |
| **7단계** | **LLM 실행** | 증강된 맥락을 바탕으로 응답 생성 | ChatOpenAI, Anthropic Claude |
| **8단계** | **체인 연결** | 전체 파이프라인을 단일 실행 체인으로 구성 | LangChain LCEL |

#### 검색 전략

- **Dense Retrieval**: 임베딩 기반 의미적 유사성 검색
- **Sparse Retrieval**: 키워드 기반 어휘적 매칭 (BM25 등)
- **Hybrid Search**: Dense와 Sparse 방법의 조합으로 검색 정확도 향상

## 시스템 최적화 포인트

### 성능 최적화

- **청크 크기 조정**: 도메인과 사용 사례에 맞는 최적 청크 크기 설정
- **검색 파라미터 튜닝**: Top-K 검색 결과 수, 유사도 임계값 조정
- **프롬프트 엔지니어링**: 맥락 활용도를 높이는 프롬프트 설계

### 품질 보증

- **출처 추적**: 응답 근거의 명확한 출처 정보 제공
- **할루시네이션 방지**: 맥락 외 정보 생성 억제 메커니즘
- **답변 일관성**: 동일 질의에 대한 일관된 응답 보장

## 실습 문서 소개

### 소프트웨어정책연구소(SPRi) AI 브리프

본 튜토리얼에서는 실제 정책 보고서를 활용하여 실무 환경과 유사한 RAG 시스템을 구현합니다.

#### 문서 정보

| 항목 | 내용 |
|------|------|
| **발행처** | 소프트웨어정책연구소(SPRi) |
| **제목** | 2025년 8월호 AI Brief |
| **저자** | 이해수AI정책연구실 선임연구원, 유재흥AI정책연구실 책임연구원 |
| **출처** | https://spri.kr/posts/view/23902 |
| **파일명** | `SPRI_AI_Brief_2025_08.pdf` |


#### 파일 구조 설정

실습을 위해 다음과 같이 파일을 배치합니다:

```
프로젝트 폴더/
├── data/
│   └── SPRI_AI_Brief_2025_08.pdf
└── 02-RAG/
    └── 00-RAG-Basic-PDF.ipynb
```

---

# 환경 설정

### API KEY 설정

RAG 시스템 구동을 위해 언어 모델 및 임베딩 서비스 API 키가 필요합니다.

In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv(override=True)  # 기존 환경변수보다 .env 파일 우선 적용

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LangChain-Tutorial")  # LangSmith 프로젝트명 설정

---

# RAG 파이프라인 구현 (1-8단계)

## 구현 개요

8단계 RAG 파이프라인을 순차적으로 구현하여 완전한 시스템을 구축합니다.

### 구현 전략

- **단계별 구현**: 각 단계를 독립적으로 구현하고 검증
- **모듈식 설계**: 각 구성 요소를 재사용 가능한 형태로 구성
- **점진적 통합**: 개별 단계를 하나의 완성된 체인으로 연결

## 필수 라이브러리

In [ ]:
# 문서 분할을 위한 텍스트 스플리터
from langchain_text_splitters import RecursiveCharacterTextSplitter

# PDF 문서 로더
from langchain_community.document_loaders import PyMuPDFLoader

# 벡터 저장소 (FAISS - 페이스북이 개발한 고속 유사도 검색 라이브러리)
from langchain_community.vectorstores import FAISS

# 출력 파서 (AI 응답을 문자열로 변환)
from langchain_core.output_parsers import StrOutputParser

# 체인 연결을 위한 패스스루 (데이터를 그대로 전달)
from langchain_core.runnables import RunnablePassthrough

# 프롬프트 템플릿
from langchain_core.prompts import PromptTemplate

# OpenAI 모델들 (ChatGPT와 임베딩 모델)
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

## RAG 시스템 기본 구조

### 단계별 구현 방식

각 단계를 개별적으로 구현하고 검증한 후 전체 파이프라인으로 통합하는 방식을 사용합니다.

#### 구현의 장점

| 장점 | 설명 |
|------|------|
| **학습 효과** | 각 단계의 역할과 중요성 명확히 이해 |
| **모듈화** | 독립적인 구성 요소로 유지보수성 향상 |
| **디버깅 용이** | 문제 발생 시 특정 단계 격리하여 해결 |
| **확장성** | 필요에 따라 개별 구성 요소 교체 가능 |

### 구현 순서

단계별로 구현하면서 각 단계의 출력을 확인하여 올바른 동작을 검증합니다.

In [ ]:
# 단계 1: 문서 로드(Load Documents)
# PDF 파일을 읽어서 텍스트로 변환하는 로더 생성
loader = PyMuPDFLoader("data/SPRI_AI_Brief_2025_08.pdf")
# 문서를 실제로 로드 (페이지별로 분리되어 저장됨)
docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")  # 총 페이지 수 출력

### 문서 내용 확인

로드된 문서가 정상적으로 처리되었는지 확인하기 위해 특정 페이지의 내용을 검토합니다.

In [ ]:
# 10번째 페이지(인덱스 9)의 내용을 출력하여 문서가 제대로 로드되었는지 확인
print(docs[9].page_content)

### 메타데이터 구조 확인

각 문서 청크에는 출처 추적을 위한 메타데이터가 포함됩니다. 이 정보는 응답 생성 시 근거 자료의 출처를 명시하는 데 활용됩니다.

In [ ]:
# 10번째 페이지 문서의 전체 정보(메타데이터 포함)를 딕셔너리 형태로 출력
# page_content: 실제 텍스트 내용, metadata: 파일명, 페이지 번호 등 부가 정보
docs[10].__dict__

In [ ]:
# 단계 2: 문서 분할(Split Documents)
# 긴 문서를 검색하기 적절한 크기로 나누는 텍스트 스플리터 생성
# chunk_size=500: 각 청크의 최대 글자 수
# chunk_overlap=50: 인접한 청크 간 겹치는 글자 수 (문맥 유지를 위함)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
# 로드된 문서들을 설정된 크기로 분할
split_documents = text_splitter.split_documents(docs)
print(f"분할된 청크의수: {len(split_documents)}")  # 분할된 총 청크 개수 출력

In [ ]:
# 단계 3: 임베딩(Embedding) 생성
# 텍스트를 벡터(숫자 배열)로 변환하는 임베딩 모델 생성
# text-embedding-3-small: OpenAI의 최신 임베딩 모델 (성능 좋고 비용 효율적)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [ ]:
# 단계 4: DB 생성(Create DB) 및 저장
# 분할된 문서들을 임베딩으로 변환하여 FAISS 벡터 데이터베이스에 저장
# FAISS: Facebook이 개발한 고속 유사도 검색 라이브러리
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [ ]:
# similarity_search: 쿼리와 가장 유사한 문서들을 반환 (기본 4개)
for doc in vectorstore.similarity_search("미드저니"):
    print(doc.page_content)  # 검색된 각 문서의 내용 출력

In [ ]:
# 단계 5: 검색기(Retriever) 생성
# 벡터 데이터베이스를 검색 가능한 retriever로 변환
# retriever는 쿼리를 받아 관련 문서를 자동으로 찾아주는 역할
retriever = vectorstore.as_retriever()

### 검색기 기능 테스트

구축된 검색기가 실제 질의에 대해 적절한 문서를 검색하는지 확인합니다.

In [ ]:
# 검색기에 질문을 넣어서 관련 문서 청크들을 검색
# invoke: 검색기를 실행하는 메서드
# 질문과 관련된 문서들을 유사도 순으로 반환
retriever.invoke("미드저니가 새롭게 출시한 모델과 출시일은?")

In [ ]:
# 단계 6: 프롬프트 생성(Create Prompt)
# AI에게 검색된 문서를 바탕으로 질문에 답변하도록 지시하는 프롬프트 템플릿
prompt = PromptTemplate.from_template(
    """You are an expert assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question accurately and comprehensively.

Instructions:
- Only use information from the provided context
- If you don't know the answer based on the context, say "죄송하지만 제공된 문서에서는 해당 정보를 찾을 수 없습니다."
- Answer in Korean with a clear and professional tone
- Cite specific parts of the context when possible

Format example for sources section:
**출처:**
- [1] filename(page number)
- [2] filename(page number)

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

In [ ]:
# 단계 7: 언어모델(LLM) 생성
import os

# 질문-답변을 수행할 ChatGPT 모델 설정
llm = ChatOpenAI(
    temperature=0.1,
    model="openai/gpt-4.1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url=os.getenv("OPENROUTER_BASE_URL"),
)

### 문서 형식화 함수

검색된 문서들을 구조화된 형태로 변환하여 LLM이 효과적으로 활용할 수 있도록 합니다.

In [ ]:
def format_docs(docs):
    return "\n".join(
        [
            f"<document><content>{doc.page_content}</content><metadata><page>{doc.metadata['page']+1}</page><source>{doc.metadata['source']}</source></metadata></document>"
            for i, doc in enumerate(docs)
        ]
    )

In [ ]:
result = retriever.invoke("미드저니가 새롭게 출시한 모델과 출시일은?")
print(format_docs(result))

In [ ]:
# 단계 8: 체인(Chain) 생성
# 검색→프롬프트→LLM→출력 파싱의 전체 파이프라인을 하나의 체인으로 연결
# {context: retriever, question: RunnablePassthrough()}: 질문을 받아 검색 결과와 함께 전달
# |: 파이프라이프 연산자로 각 단계를 순차적으로 연결
chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),
    }  # 질문을 검색기에 전달하고 결과를 context로 설정
    | prompt  # 검색 결과와 질문을 프롬프트 템플릿에 삽입
    | llm  # 완성된 프롬프트를 LLM에 전달하여 답변 생성
    | StrOutputParser()  # LLM 응답을 문자열로 변환
)

### RAG 시스템 실행

완성된 RAG 체인을 실행하여 전체 파이프라인의 동작을 확인합니다.

#### 실행 과정

1. **질의 입력**: 사용자 질문 처리
2. **문서 검색**: 관련 문서 청크 자동 검색
3. **맥락 구성**: 검색된 문서와 질의를 결합
4. **응답 생성**: LLM을 통한 최종 답변 생성

모든 단계가 자동으로 순차 실행됩니다.

In [ ]:
# 체인 실행(Run Chain)
# 완성된 RAG 파이프라인에 질문을 입력하여 실행
question = "미드저니가 새롭게 출시한 모델과 출시일은?"
# invoke: 체인을 실행하는 메서드 (질문 → 검색 → 프롬프트 → LLM → 답변)
response = chain.invoke(question)
print(response)  # 최종 답변 출력

---

# 통합 RAG 시스템

## 완전한 구현 코드

### 프로덕션 준비 코드

지금까지 단계별로 구현한 내용을 하나의 완전한 시스템으로 통합합니다.

#### 최적화된 설정

| 설정 항목 | 값 | 목적 |
|----------|---|------|
| **chunk_size** | 1000 | 충분한 맥락 정보 포함 |
| **chunk_overlap** | 50 | 청크 간 연속성 보장 |
| **embedding_model** | text-embedding-3-small | 성능-비용 최적화 |
| **temperature** | 0.1 | 일관된 응답 생성 |

#### 시스템 특징

- **재사용 가능**: 다른 PDF 문서에 즉시 적용 가능
- **모듈식 구조**: 각 구성 요소 독립적 교체 가능
- **확장 가능**: 추가 기능 통합 용이
- **운영 환경 적합**: 실제 서비스에서 바로 사용 가능

In [ ]:
# 필요한 라이브러리 import
import os
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader("data/SPRI_AI_Brief_2025_08.pdf")
docs = loader.load()

# 단계 2: 문서 분할(Split Documents)
# chunk_size를 1000으로 설정하여 더 많은 맥락 정보 포함
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)

# 단계 3: 임베딩(Embedding) 생성
# text-embedding-3-small 모델 사용 (성능과 비용의 최적 균형)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 단계 4: DB 생성(Create DB) 및 저장
# FAISS 벡터 데이터베이스 생성 및 문서 저장
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

# 단계 5: 검색기(Retriever) 생성
# 질의와 유사한 문서를 검색하는 retriever 생성
retriever = vectorstore.as_retriever()

# 단계 6: 프롬프트 생성(Create Prompt)
# 개선된 프롬프트 템플릿으로 더 정확한 답변 유도
prompt = PromptTemplate.from_template(
    """You are an expert assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question accurately and comprehensively.

Instructions:
- Only use information from the provided context
- If you don't know the answer based on the context, say "죄송하지만 제공된 문서에서는 해당 정보를 찾을 수 없습니다."
- Answer in Korean with a clear and professional tone
- Cite specific parts of the context when possible

Format example for sources section(Only if your answer is based on the context):
**출처:**
- [1] filename(page number)
- [2] filename(page number)

#Context: 
{context}

#Question:
{question}

#Answer:"""
)

# 단계 7: 언어모델(LLM) 생성
# GPT 모델 사용, temperature=0으로 일관된 답변 생성
llm = ChatOpenAI(
    temperature=0.1,
    model="openai/gpt-4.1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url=os.getenv("OPENROUTER_BASE_URL"),
)

# 단계 8: 체인(Chain) 생성
# 전체 RAG 파이프라인을 하나의 실행 가능한 체인으로 구성
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# 체인 실행(Run Chain) - 최종 테스트
# RAG 시스템에 질문을 입력하여 완전한 파이프라인 실행
question = "미드저니가 새롭게 출시한 모델과 출시일은?"
# 질문 → 문서검색 → 프롬프트생성 → AI답변 → 결과반환의 전체 과정이 자동 실행
response = chain.invoke(question)
print(response)